# Profiling ANDES in Notebook

## Profiling with Python CProfiler

In [1]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

Passing `profile=True, no_output = True` to `run` will enable the profiler and have the results printed.

In [2]:
ss = andes.run(case_path, profile=True, routine='tds', no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0923 second.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0109 second.

-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0sec.
Variable step size: h0=33.33msec.
Initialization was successful in 0.0223 second.


<Toggle 1>: Status of Line.Line_8 changed to 0.0 at t=2.0sec.         
100%|████████████████████████████████| 100/100 [00:02<00:00, 36.42%/s]

Simulation completed in 2.7460 seconds.



-> Single process finished in 2.9945 seconds.


         1751864 function calls (1743425 primitive calls) in 2.974 seconds

   Ordered by: cumulative time
   List reduced from 2013 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.007    0.007    2.812    2.812 /Users/hcui7/repos/andes/andes/routines/tds.py:138(run)
      600    0.106    0.000    2.708    0.005 /Users/hcui7/repos/andes/andes/routines/tds.py:233(_implicit_step)
    11036    0.018    0.000    1.955    0.000 /Users/hcui7/repos/andes/andes/system.py:716(_call_models_method)
    11036    0.143    0.000    1.931    0.000 /Users/hcui7/repos/andes/andes/system.py:727(<listcomp>)
     1834    0.001    0.000    0.949    0.001 /Users/hcui7/repos/andes/andes/system.py:445(g_update)
    53131    0.480    0.000    0.902    0.000 /Users/hcui7/repos/andes/andes/core/model.py:1202(g_update)
     1834    0.001    0.000    0.564    0.000 /Users/hcui7/repos/andes/andes/system.py:435(f_update)
    53131    0.368    0.00

## Profiling with `line_profiler`.

`line_profiler` provides line-based profiling results for functions. 

Install with `pip install line_profiler` and restart the notebook.

In [3]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

### Profile power flow 

Pass the function name to profile to the magic `%lprun`, followed by a call to the function itself or an upper-level function.

Results will be shown in a popup window.

In [4]:
%lprun -f andes.routines.pflow.PFlow.run andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.1180 second.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0121 second.
-> Single process finished in 0.1874 second.


Alternatively, do

In [5]:
ss = andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0591 second.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0096 second.
-> Single process finished in 0.1028 second.


In [6]:
%lprun -f ss.PFlow.run ss.PFlow.run()


-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0115 second.


To dig into the Newton Raphson iteration steps, profile each step instead with:

In [7]:
%lprun -f ss.PFlow.nr_step ss.PFlow.run()


-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0115 second.


### Profile time-domain simulation

In [8]:
%lprun -f ss.TDS._implicit_step ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0sec.
Variable step size: h0=33.33msec.
Initialization was successful in 0.0251 second.


<Toggle 1>: Status of Line.Line_8 changed to 0.0 at t=2.0sec.         
100%|████████████████████████████████| 100/100 [00:03<00:00, 33.03%/s]

Simulation completed in 3.0284 seconds.


## Cleanup

In [9]:
!andes misc -C


    _           _         | Version 0.8.5.post39.dev0+ga1b7093a
   /_\  _ _  __| |___ ___ | Python 3.7.1 on Darwin, 04/23/2020 07:53:29 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

No output file found in the working directory.
